In [1]:
import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [ ]:
paths = ["data/normal", "data/codiv19"]  
imgs = []  
labels = [] 

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 0)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy
imgs = np.array(imgs).reshape(-1, 150, 150, 3)  # Añadir dimensión de canal para imágenes en escala de grises
labels = np.array(labels)

print(f"Images shape: {imgs.shape}")
print(f"Num. of labels: {len(labels)}")

Images shape: (30, 150, 150, 3)
Num. of labels: 30


In [3]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"), 
    keras.layers.MaxPooling2D(pool_size=(2, 2)),  

    keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Conv2D(128, (3, 3), padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Flatten(),  
    
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),  

    keras.layers.Dense(2, activation="softmax") 
])

# Imprimir el resumen del modelo
print(model.summary())

c:\Users\agusb\OneDrive\Escritorio\ia\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 148, 148, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 74, 74, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     5,308,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,410,722 (20.64 MB)

 Trainable params: 5,410,722 (20.64 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    imgs, labels,
    epochs=20,  
    batch_size=32,
    verbose=1
)

Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv2d" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, 150, 150, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 150, 150, 3), dtype=float32)
  • training=True
  • mask=None

In [ ]:
#from keras.models import load_model
#model.save('modelo.keras')

# El siguiente es el que se convertira con tensorflowjs
model.save('modelo.h5')

In [ ]:
#Cargar el modelo ya entrenado con extensión .keras
#modelo_cargado = load_model('modelo.keras')

In [ ]:

# Función para cargar y preprocesar una imagen
def load_and_preprocess_image(image_path):
    # Cargar la imagen
    img = cv2.imread(image_path, 3)  # 0 para leer en escala de grises

    # Convertir la imagen a escala de grises si tiene 3 canales (RGB)
    #if len(img.shape) == 3:
     #   img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Redimensionar la imagen a (150, 150)
    img = cv2.resize(img, (150, 150))
    # Normalizar la imagen
    img = img / 255.0
    # Añadir una dimensión para los canales (la imagen debe tener forma (150, 150, 1))
    img = np.expand_dims(img, axis=-1)
    # Añadir una dimensión para que la forma sea (1, 150, 150, 1)
    img = np.expand_dims(img, axis=0)
    return img

# Función para hacer predicciones
def predict_image(model, image_path):
    # Cargar y preprocesar la imagen
    img = load_and_preprocess_image(image_path)
    # Realizar la predicción
    predictions = model.predict(img)
    # Obtener la clase con mayor probabilidad
    predicted_class = np.argmax(predictions)

    # Obtener las probabilidades de cada clase
    probabilities = predictions[0]  # predictions es un array de forma (1, 2)

    # Imprimir el resultado con probabilidades
    print(f"Probabilidad de normal: {probabilities[0]:.4f}")
    print(f"Probabilidad de COVID-19: {probabilities[1]:.4f}")

    # Imprimir el resultado
    if predicted_class == 0:
        print("La imagen es de una persona normal (sin COVID-19).")
    else:
        print("La imagen es de una persona con COVID-19.")




In [ ]:
# Ejemplo de uso:
image_path = 'data/normal/N25.jpg'
predict_image(model, image_path)